In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# BaristaBot: AI-Powered Cafe Ordering System

## The Problem: Streamlining Cafe Ordering Experiences

Traditional cafe ordering systems face several challenges:

- They often lack the natural conversational flow that customers prefer
- Menu information may be scattered or difficult to search
- Ordering complex drinks with multiple modifiers can be confusing
- Order confirmation and validation are frequently manual processes
- Customer service quality can be inconsistent depending on staff availability

## How Generative AI Solves This

This notebook demonstrates a powerful solution through **BaristaBot**, an interactive cafe ordering system powered by:

1. **LangGraph** for creating a multi-step conversational workflow
2. **Google's Gemini model** for natural language understanding
3. **Vector databases** for intelligent menu search and retrieval

The system showcases how generative AI can:

- Engage in natural dialog about menu items and their details
- Maintain contextual awareness throughout the ordering process
- Dynamically search and retrieve menu information using semantic understanding
- Process complex orders with modifiers (milk types, sweeteners, temperature preferences)
- Validate its own responses for accuracy before presenting to customers
- Confirm orders explicitly with customers before processing

This implementation demonstrates how AI can transform customer service in food service settings by combining the convenience of digital ordering with the natural feel of human conversation, all while maintaining accuracy and consistency.

## Project Overview

This capstone project enhances the original BaristaBot from the 5-Day Gen AI Intensive Course by implementing advanced AI techniques to create a more intelligent and responsive cafe ordering system.

## Original Source & Attribution

- **Course**: [5-Day Gen AI Intensive Course with Google Learn Guide](https://www.kaggle.com/learn-guide/5-day-genai)
- **Original Code**: [Day 3 - Building an Agent with LangGraph](https://www.kaggle.com/code/markishere/day-3-building-an-agent-with-langgraph/)
- **Course Developers**: Addison Howard, Brenda Flynn, Myles O'Neill, Nate, and Polong Lin
- **Competition**: [Gen AI Intensive Course Capstone 2025Q1](https://kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1)

## Technical Enhancements & Implementation

The system leverages these core technologies and improvements:

### 1. Vector Database with FAISS
- Replaces hardcoded menu with dynamic, searchable vector database
- Enables semantic search across menu items and categories
- Provides scalability for menu updates without code changes

### 2. Retrieval Augmented Generation (RAG)
- Dynamically retrieves relevant menu information based on customer queries
- Delivers contextual responses about drink options and modifiers
- Supports natural language search for menu information

### 3. LLM-Based Quality Control
- Evaluates responses before presentation to customers
- Automatically corrects responses that don't meet quality standards
- Ensures consistent adherence to menu options and clear communication

## Project Significance

This implementation demonstrates how combining LangGraph, Gemini AI, vector search, and evaluation systems can transform customer service applications by creating more accurate, natural, and adaptable conversational experiences in real-world business settings.

## Get set up

Start by installing and importing the LangGraph SDK and LangChain support for the Gemini API.

In [2]:
# Remove conflicting packages from the Kaggle base environment.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
# Install langgraph and the packages used in this lab.
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7' 'langchain_community' 'faiss-cpu'

### Set up your API key

The `GOOGLE_API_KEY` environment variable can be set to automatically configure the underlying API. This works for both the official Gemini Python SDK and for LangChain/LangGraph. 

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [3]:
import os
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [4]:
from typing import Annotated
from typing_extensions import TypedDict
from typing import Literal
from typing import Iterable

from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END

from langchain_core.messages.ai import AIMessage
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.tools import tool
from langchain_core.messages.tool import ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI

from collections.abc import Iterable
from random import randint

from pprint import pprint, PrettyPrinter

# State Management in BaristaBot

In this section, we define the core state structure and system instructions for our BaristaBot application. Let's examine how LangGraph manages conversation state:

## OrderState Class

This class is the foundation of our application. Let's break down each component:

### TypedDict
- This Python feature provides static type hints for dictionaries with a fixed set of keys, making our code more maintainable.

### messages
- Stores the entire conversation history
- The `Annotated[list, add_messages]` syntax is crucial - it tells LangGraph to append new messages rather than replacing the entire list
- Without this annotation, each node would override previous messages instead of building a conversation

### order
- A simple list of strings representing ordered items
- Each string contains both the drink and its modifiers (e.g., "Latte (oat milk, vanilla)")

### finished
- Boolean flag indicating when to terminate the conversation
- Set to True when order is placed or user quits

## The BARISTABOT_SYSINT tuple 

This tuple contains system instructions that define the LLM's behavior:

### Key aspects of these instructions:

#### Role Definition
- Establishes the bot's identity and purpose

#### Conversation Boundaries
- Restricts discussion to menu items and products

#### Tool Usage Guidelines
- Defines when and how to use various functions:
  * `get_menu` and `search_menu` for accessing menu information
  * `add_to_order`, `clear_order`, `get_order` for order management
  * `confirm_order` for verification
  * `place_order` for completion

#### Quality Standards
- Mentions the evaluation process for responses

#### Fallback Behavior
- Instructions for handling unavailable tools

## The welcome message 

`WELCOME_MSG` provides the initial greeting and exit instructions.

## Why This Matters

This approach to state management offers several benefits:
* **Persistence**: Conversation history is maintained across different processing nodes
* **Modularity**: Separating state from logic makes the system easier to extend
* **Clarity**: Type hints improve code readability and help catch errors early

In [5]:
class OrderState(TypedDict):
    """State representing the customer's order conversation."""

    # The chat conversation. This preserves the conversation history
    # between nodes. The `add_messages` annotation indicates to LangGraph
    # that state is updated by appending returned messages, not replacing
    # them.
    messages: Annotated[list, add_messages]

    # The customer's in-progress order.
    order: list[str]

    # Flag indicating that the order is placed and completed.
    finished: bool


# The system instruction defines how the chatbot is expected to behave and includes
# rules for when to call different functions, as well as rules for the conversation, such
# as tone and what is permitted for discussion.
BARISTABOT_SYSINT = (
    "system",  # 'system' indicates the message is a system instruction.
    "You are a BaristaBot, an interactive cafe ordering system. A human will talk to you about the "
    "available products you have and you will answer any questions about menu items (and only about "
    "menu items - no off-topic discussion, but you can chat about the products and their history). "
    "The customer will place an order for 1 or more items from the menu, which you will structure "
    "and send to the ordering system after confirming the order with the human. "
    "\n\n"
    "You have access to an up-to-date menu database. Use get_menu to retrieve the full menu, and "
    "search_menu with a specific query to find detailed information about particular items, modifiers, "
    "or categories. Always use these tools when you need menu information rather than relying on your "
    "general knowledge about coffee drinks."
    "\n\n"
    "Add items to the customer's order with add_to_order, and reset the order with clear_order. "
    "To see the contents of the order so far, call get_order (this is shown to you, not the user) "
    "Always confirm_order with the user (double-check) before calling place_order. Calling confirm_order will "
    "display the order items to the user and returns their response to seeing the list. Their response may contain modifications. "
    "Always verify and respond with drink and modifier names from the MENU before adding them to the order. "
    "If you are unsure a drink or modifier matches those on the MENU, use search_menu to confirm availability before adding to the order. "
    "You only have the modifiers listed on the menu. "
    "Once the customer has finished ordering items, Call confirm_order to ensure it is correct then make "
    "any necessary updates and then call place_order. Once place_order has returned, thank the user and "
    "say goodbye!"
    "\n\n"
    "Remember that your responses will be evaluated before reaching the user to ensure they: "
    "1. Correctly follow menu options "
    "2. Use appropriate tools for ordering "
    "3. Stay on-topic about cafe items only "
    "4. Are helpful and clear for customers "
    "\n\n"
    "If any of the tools are unavailable, you can break the fourth wall and tell the user that "
    "they have not implemented them yet and should keep reading to do so.",
)

# This is the message with which the system opens the conversation.
WELCOME_MSG = "Welcome to the BaristaBot cafe. Type `q` to quit. How may I serve you today?"

## LLM Initialization:

We create an instance of the Google Generative AI chat model
Specifically, we're using gemini-2.0-flash, Google's optimized model for fast responses


In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")


## Understanding the human_node Function

### Function Purpose
The human_node function is responsible for handling the human side of the conversation in this BaristaBot application. It's a critical component that:
* Shows the AI's message to the user
* Collects the user's response
* Processes any exit commands
* Updates the conversation state

### Function Parameters and Return Value
* Input: Takes a parameter `state` of type `OrderState` (a TypedDict that stores conversation history and order information)
* Output: Returns an updated `OrderState` with the user's new message added

In [7]:
def human_node(state: OrderState) -> OrderState:
    """Display the last model message to the user, and receive the user's input."""
    last_msg = state["messages"][-1]
    print("Model:", last_msg.content)

    user_input = input("User: ")

    # If it looks like the user is trying to quit, flag the conversation
    # as over.
    if user_input in {"q", "quit", "exit", "goodbye"}:
        state["finished"] = True

    return state | {"messages": [("user", user_input)]}




## Understanding the maybe_exit_human_node Function

This function serves as a crucial routing decision point in the BaristaBot conversation flow:

### Function Purpose
The maybe_exit_human_node function determines where the conversation should flow after the user has provided input. It acts as a conditional router that decides whether to:
* Continue the conversation by proceeding to the chatbot node
* End the conversation entirely

### Function Signature
* Input: Takes a parameter `state` of type `OrderState`
* Output: Returns a string literal that can be either "chatbot" or "__end__" (represented by the constant `END`)

### Decision Logic
The function contains a simple conditional check:
```python
if state.get("finished", False):
    return END
else:
    return "chatbot"
```

This checks if the `finished` flag in the state dictionary is True:
* If True, the function returns `END`, which signals to LangGraph that the conversation should terminate
* If False, it returns "chatbot", which routes the flow to the chatbot node to generate the next AI response

The `.get("finished", False)` method safely retrieves the value of the "finished" key with a default of False if the key doesn't exist.

### In the LangGraph Context
In the overall application, this function is used as a conditional edge in the state graph:
```python
graph_builder.add_conditional_edges("human", maybe_exit_human_node)
```

This tells LangGraph to use the return value of `maybe_exit_human_node` to determine where to route the state after it passes through the `human_node`. The routing is based on the value of the `finished` flag, which is set to True in `human_node` when the user types an exit command like "q", "quit", "exit", or "goodbye".

In [8]:
def maybe_exit_human_node(state: OrderState) -> Literal["chatbot", "__end__"]:
    """Route to the chatbot, unless it looks like the user is exiting."""
    if state.get("finished", False):
        return END
    else:
        return "chatbot"

In [9]:
# The default recursion limit for traversing nodes is 25 - setting it higher means
# you can try a more complex order with multiple steps and round-trips (and you
# can chat for longer!)
config = {"recursion_limit": 100}


## Building a Vector Database for the Cafe Menu

BaristaBot currently has no awareness of the available items at the cafe, so it will hallucinate a menu. One option would be to hard-code a menu into the system prompt. A better option would be to provide searchable vector database that can easily be updated.
        
This code creates a searchable vector database containing the cafe's menu items, allowing the BaristaBot to quickly find relevant menu information based on natural language queries:

### What This Function Does

The `initialize_menu_db()` function creates a searchable knowledge base of the cafe's menu by:

1. **Structuring the menu as documents**:
   Each menu section becomes a Document object with:
   * `page_content`: The actual text describing menu items
   * `metadata`: Categorization information (like "Coffee", "Tea", "Modifiers")

2. **Organizing menu information into categories**:
   * Basic drink types (Coffee, Tea, Other)
   * Drinks with milk additions
   * Available modifiers (milk types, espresso shots, sweeteners)
   * Special terminology and rules
   * Detailed descriptions of popular drinks

3. **Creating vector embeddings**: Using Google's AI embeddings model to convert menu text into numerical vector representations that capture semantic meaning

4. **Building a FAISS vector store**: Creating an efficient similarity search database that can find the most relevant menu items based on natural language queries

### Why This Matters

This vector database enables the BaristaBot to:
* Answer questions about what drinks are available
* Find specific information about drink ingredients and preparation
* Understand what customization options are available
* Process natural language queries without requiring exact keyword matching
* Maintain an up-to-date menu without hardcoding it into the application logic

The vector store is queried by the `get_menu()` and `search_menu()` tools later in the application, allowing the AI assistant to provide accurate and relevant menu information to customers.

In [10]:
# Create a global vector database for the menu
def initialize_menu_db():
    # Split menu into documents - one for each section/item
    menu_docs = [
        Document(page_content="Coffee Drinks: Espresso, Americano, Cold Brew", 
                 metadata={"category": "Coffee"}),
        Document(page_content="Coffee Drinks with Milk: Latte, Cappuccino, Cortado, Macchiato, Mocha, Flat White", 
                 metadata={"category": "Coffee with Milk"}),
        Document(page_content="Tea Drinks: English Breakfast Tea, Green Tea, Earl Grey", 
                 metadata={"category": "Tea"}),
        Document(page_content="Tea Drinks with Milk: Chai Latte, Matcha Latte, London Fog", 
                 metadata={"category": "Tea with Milk"}),
        Document(page_content="Other Drinks: Steamer, Hot Chocolate", 
                 metadata={"category": "Other"}),
        Document(page_content="Milk options: Whole, 2%, Oat, Almond, 2% Lactose Free; Default option: whole", 
                 metadata={"category": "Modifiers"}),
        Document(page_content="Espresso shots: Single, Double, Triple, Quadruple; default: Double", 
                 metadata={"category": "Modifiers"}),
        Document(page_content="Caffeine: Decaf, Regular; default: Regular", 
                 metadata={"category": "Modifiers"}),
        Document(page_content="Hot-Iced: Hot, Iced; Default: Hot", 
                 metadata={"category": "Modifiers"}),
        Document(page_content="Sweeteners: vanilla sweetener, hazelnut sweetener, caramel sauce, chocolate sauce, sugar free vanilla sweetener", 
                 metadata={"category": "Modifiers"}),
        Document(page_content="Special terms: 'dirty' means add a shot of espresso to a drink that doesn't usually have it", 
                 metadata={"category": "Special"}),
        Document(page_content="Milk terms: 'Regular milk' is the same as 'whole milk'", 
                 metadata={"category": "Special"}),
        Document(page_content="Sweetener terms: 'Sweetened' means add regular sugar, not a flavored sweetener",
                 metadata={"category": "Special"}),
        Document(page_content="Availability: Soy milk has run out of stock today, so soy is not available",
                 metadata={"category": "Special"}),
        # Add detailed descriptions for popular drinks
        Document(page_content="Latte: Espresso with steamed milk and a light layer of foam. Flavors can be added.",
                 metadata={"category": "Drink Details", "item": "Latte"}),
        Document(page_content="Cappuccino: Equal parts espresso, steamed milk, and foam for a stronger coffee taste.",
                 metadata={"category": "Drink Details", "item": "Cappuccino"}),
        Document(page_content="Chai Latte: Spiced black tea concentrate mixed with steamed milk.",
                 metadata={"category": "Drink Details", "item": "Chai Latte"}),
    ]
    
    # Create embeddings and vector store
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    return FAISS.from_documents(menu_docs, embeddings)

In [11]:
# Initialize the vector store once
menu_db = initialize_menu_db()

## The get_menu() Tool Function Explained

This function creates a structured menu display by retrieving and organizing menu items from the vector database:

### Function Overview

The `get_menu()` function is labeled with the `@tool` decorator, which registers it as a callable tool that the LLM can use during conversations with customers. This function retrieves and formats the entire cafe menu in a readable format.

### Key Components

1. **Function Decoration**:
   - The `@tool` decorator makes this function available to the LLM as a named tool that can be invoked during conversation

2. **Vector Database Query**:
   - `retriever = menu_db.as_retriever(search_kwargs={"k": 10})` creates a retriever that will return up to 10 matching documents
   - `docs = retriever.invoke("all menu categories")` queries the database for menu items across all categories

3. **Menu Organization**:
   - The function initializes an empty dictionary to group menu items by category
   - It iterates through each retrieved document and organizes them by their category metadata
   - If a category doesn't exist in the dictionary yet, it creates a new list for that category

4. **Menu Formatting**:
   - The function builds a well-formatted string containing the entire menu
   - Items are grouped by category, making the menu easier to read and understand
   - Each category's items are joined with newlines and separated by blank lines

### Why This Matters

This function serves as the BaristaBot's knowledge source for the complete menu, allowing it to:
* Provide customers with the full menu when requested
* Answer general questions about what's available
* Stay up-to-date with menu changes (as the vector database can be updated)
* Present menu information in a structured, user-friendly format

When a customer asks something like "What do you have?" or "Can I see the menu?", the LLM can call this function to retrieve the complete, organized menu to share with the customer.

In [12]:
@tool
def get_menu() -> str:
    """Provide the latest up-to-date menu from our menu database."""
    # Get all menu categories
    retriever = menu_db.as_retriever(search_kwargs={"k": 10})
    # Use invoke instead of get_relevant_documents
    docs = retriever.invoke("all menu categories")
    
    # Build a formatted menu from retrieved documents
    menu_text = "MENU:\n"
    
    # Group documents by category
    categories = {}
    for doc in docs:
        category = doc.metadata.get("category", "Other")
        if category not in categories:
            categories[category] = []
        categories[category].append(doc.page_content)
    
    # Format the menu
    for category, items in categories.items():
        menu_text += "\n".join(items) + "\n\n"
        
    return menu_text

## The search_menu Tool Function Explained

This function creates a specialized menu search capability that lets the BaristaBot find specific information:

### Function Purpose
While `get_menu()` retrieves the entire menu, `search_menu()` allows for targeted searches to answer specific customer questions about menu items, ingredients, or options.

### Key Components
1. **Tool Registration**:
   - The `@tool` decorator registers this function as a tool that the LLM can invoke during conversations
   - The function signature shows it takes a string query and returns a string result

2. **Focused Retrieval**:
   - `retriever = menu_db.as_retriever(search_kwargs={"k": 3})` creates a retriever limited to the 3 most relevant results
   - This prevents overwhelming customers with too much information

3. **Semantic Search**:
   - `docs = retriever.invoke(query)` performs a semantic search using the query string
   - This allows natural language questions like "What milk options do you have?" or "Tell me about your lattes"

4. **Error Handling**:
   - The function handles the case where no relevant information is found
   - It returns a clear message stating nothing was found for the query

5. **Result Formatting**:
   - The function formats the results with a clear header
   - It includes all the content from each matching document in the response

### Why This Matters
This function enables more natural conversation by allowing BaristaBot to:
* Answer specific questions about menu items ("What's in a chai latte?")
* Find available options for customization ("What milk alternatives do you have?")
* Check if special requests are possible ("Can I get a decaf cappuccino?")
* Provide detailed information without overwhelming the customer with the entire menu

This targeted search capability makes the BaristaBot more helpful and efficient, allowing customers to get exactly the information they need about specific menu items or options.

In [13]:
# Add an additional search_menu tool
@tool
def search_menu(query: str) -> str:
    """Search for specific information in the menu database."""
    retriever = menu_db.as_retriever(search_kwargs={"k": 3})
    # Use invoke instead of get_relevant_documents
    docs = retriever.invoke(query)
    
    if not docs:
        return f"I couldn't find information about '{query}' in our menu."
    
    result = f"Here's what I found about '{query}':\n\n"
    for doc in docs:
        result += doc.page_content + "\n"
    
    return result

## Empty Tool Functions vs. Implementation

This code section showcases an important LangGraph pattern where tool interfaces are defined separately from their implementations:

### The Pattern: Empty Function Bodies

These functions are intentionally defined with empty bodies because of a specific architectural constraint in LangGraph:

* **Tool Registration**: The `@tool` decorator registers these functions as available tools for the LLM to call
* **State Management Limitation**: LangGraph doesn't allow `@tool`-decorated functions to directly modify the conversation state
* **Deferred Implementation**: The actual implementation of these functions is handled separately in the `order_node` function

### Why This Approach Matters

This pattern creates a clean separation between:

1. **Tool Definition**: The function signatures and docstrings create a clear interface that tells the LLM:
   * What tools are available
   * What parameters they take
   * What they return
   * What they're supposed to do

2. **Tool Implementation**: The actual behavior is implemented elsewhere (in the `order_node`), where it can properly update the state

This separation is beneficial because:
* It provides a clean API for the LLM to understand and use
* It centralizes state-changing logic in dedicated nodes
* It follows LangGraph's constraints around state management

### The Order Management Tools

The five defined tools cover the complete order lifecycle:
* `add_to_order`: Adds drinks with customizations to the order
* `get_order`: Retrieves the current order contents
* `confirm_order`: Asks the customer if the order is correct
* `clear_order`: Removes all items from the order
* `place_order`: Finalizes the order and provides wait time

Together, these tools enable the BaristaBot to manage the ordering process while maintaining a clean architectural separation between tool definition and state-changing implementation.

In [14]:
# These functions have no body; LangGraph does not allow @tools to update
# the conversation state, so you will implement a separate node to handle
# state updates. Using @tools is still very convenient for defining the tool
# schema, so empty functions have been defined that will be bound to the LLM
# but their implementation is deferred to the order_node.


@tool
def add_to_order(drink: str, modifiers: Iterable[str]) -> str:
    """Adds the specified drink to the customer's order, including any modifiers.

    Returns:
      The updated order in progress.
    """


@tool
def confirm_order() -> str:
    """Asks the customer if the order is correct.

    Returns:
      The user's free-text response.
    """


@tool
def get_order() -> str:
    """Returns the users order so far. One item per line."""


@tool
def clear_order():
    """Removes all items from the user's order."""


@tool
def place_order() -> int:
    """Sends the order to the barista for fulfillment.

    Returns:
      The estimated number of minutes until the order is ready.
    """

## The order_node Function: Implementing Order Management Logic

The `order_node` function is the core implementation of the BaristaBot's order management capabilities:

### Function Purpose
This function provides the actual implementation for all the ordering tools that were previously defined with empty bodies. When the LLM decides to use an ordering tool, this function executes the appropriate logic and updates the conversation state.

### Key Implementation Details

1. **State Management**
   * The function retrieves the current state (latest message and order list)
   * It tracks whether an order has been placed
   * It collects messages to be sent back

2. **Tool Implementation**
   For each tool call, the function implements specific behavior:

   #### add_to_order
   * Formats drink items with their modifiers (e.g., "Latte (oat milk, vanilla)")
   * Adds the formatted item to the order list
   * Returns the updated order as a string

   #### confirm_order
   * Displays the current order items directly to the customer
   * Explicitly shows empty orders as "(no items)"
   * Collects confirmation directly from the customer via input
   * Important: Shows the exact order data to avoid AI hallucination issues

   #### get_order
   * Returns the formatted order or "(no order)" if empty
   * Used by the LLM to check what's in the order without showing the customer

   #### clear_order
   * Empties the order list
   * Returns no response (None)

   #### place_order
   * Displays the final order that's being sent to the kitchen
   * Sets the order_placed flag to true
   * Returns a random estimated wait time (1-5 minutes)

3. **Response Handling**
   * Creates a `ToolMessage` for each tool response
   * Includes proper tool call IDs for response tracking
   * Returns the updated state with new messages, order list, and completion status

### Architecture Significance
This implementation demonstrates a key LangGraph pattern where:
* Tools are defined with the `@tool` decorator (providing interface and documentation)
* The actual implementation lives in a dedicated node that handles state updates
* The results are formatted as tool messages that can be processed by the LLM

This separation maintains a clean architecture while allowing the LLM to use tools that modify the application state.

In [15]:
def order_node(state: OrderState) -> OrderState:
    """The ordering node. This is where the order state is manipulated."""
    tool_msg = state.get("messages", [])[-1]
    order = state.get("order", [])
    outbound_msgs = []
    order_placed = False

    for tool_call in tool_msg.tool_calls:

        if tool_call["name"] == "add_to_order":

            # Each order item is just a string. This is where it assembled as "drink (modifiers, ...)".
            modifiers = tool_call["args"]["modifiers"]
            modifier_str = ", ".join(modifiers) if modifiers else "no modifiers"

            order.append(f'{tool_call["args"]["drink"]} ({modifier_str})')
            response = "\n".join(order)

        elif tool_call["name"] == "confirm_order":

            # We could entrust the LLM to do order confirmation, but it is a good practice to
            # show the user the exact data that comprises their order so that what they confirm
            # precisely matches the order that goes to the kitchen - avoiding hallucination
            # or reality skew.

            # In a real scenario, this is where you would connect your POS screen to show the
            # order to the user.

            print("Your order:")
            if not order:
                print("  (no items)")

            for drink in order:
                print(f"  {drink}")

            response = input("Is this correct? ")

        elif tool_call["name"] == "get_order":
            response = "\n".join(order) if order else "(no order)"

        elif tool_call["name"] == "clear_order":

            order.clear()
            response = None

        elif tool_call["name"] == "place_order":

            order_text = "\n".join(order)
            print("Sending order to kitchen!")
            print(order_text)

            # TODO(you!): Implement cafe.
            order_placed = True
            response = randint(1, 5)  # ETA in minutes

        else:
            raise NotImplementedError(f'Unknown tool call: {tool_call["name"]}')

        # Record the tool results as tool messages.
        outbound_msgs.append(
            ToolMessage(
                content=response,
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )

    return {"messages": outbound_msgs, "order": order, "finished": order_placed}


# The maybe_route_to_tools Function: Dynamic Conversation Routing

This function serves as the central traffic controller for the BaristaBot conversation flow, determining where to route the conversation based on the current state.

## Function Purpose
The function analyzes the current conversation state and dynamically decides where to route the flow next. It examines the last message and routes to the appropriate node based on the message content and order status.

## Routing Logic Breakdown
1. **Initial Validation**
   - Checks if messages exist in the state
   - Raises an error if no messages are found
   - Retrieves the last message for analysis

2. **Exit Condition Check**
   - Checks if the order has been completed (state.finished is True)
   - Returns the special END constant to terminate the conversation
   - Relies on the AI's system instructions to provide a proper goodbye message

3. **Tool Call Detection**
   - Checks if the message contains tool calls (AI requesting to use tools)
   - Distinguishes between two types of tools:
     - Automated tools (menu-related): Routes to the "tools" node
     - Order management tools: Routes to the "ordering" node
   - This separation allows different handling for information retrieval vs. state-changing operations

4. **Default Human Interaction**
   - If no special routing is needed, routes to the "human" node
   - This allows the human user to respond to the AI's message

## In the LangGraph Context
This function is used as a conditional edge in the state graph. The dynamic routing creates a conversation flow that allows BaristaBot to:
- Retrieve menu information when needed
- Manage the order state properly
- Interact with the user naturally
- Exit gracefully when the order is complete

This routing mechanism enables the conversation to flow naturally between different components while maintaining appropriate state transitions throughout the ordering process.

In [16]:
def maybe_route_to_tools(state: OrderState) -> str:
    """Route between chat and tool nodes if a tool call is made."""
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    msg = msgs[-1]

    if state.get("finished", False):
        # When an order is placed, exit the app. The system instruction indicates
        # that the chatbot should say thanks and goodbye at this point, so we can exit
        # cleanly.
        return END

    elif hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        # Route to `tools` node for any automated tool calls first.
        if any(
            tool["name"] in tool_node.tools_by_name.keys() for tool in msg.tool_calls
        ):
            return "tools"
        else:
            return "ordering"

    else:
        return "human"


# The chatbot_with_tools Function: Core LLM Interaction Node

## Function Purpose
This function serves as the primary interaction point with the language model in the BaristaBot conversation system. It processes the current state, generates AI responses, and maintains conversation continuity.

## Key Features

1. **State Handling**
   * Receives the current OrderState containing conversation history and order details
   * Establishes default values for "order" and "finished" to ensure state consistency
   * Returns an updated state with new AI messages

2. **Response Generation Logic**
   * For ongoing conversations: Invokes the language model with both system instructions and conversation history
   * For new conversations: Creates an initial welcome message
   * Uses tool-enhanced LLM (llm_with_tools) that has access to menu and ordering functions

## State Management
* Preserves existing state properties while updating only the "messages" field
* Uses Python's dictionary merging (| operator) to combine defaults, current state, and new messages
* Ensures the state structure remains consistent throughout the conversation flow

## In the LangGraph Context
This function operates as a node in the state graph that:
* Processes incoming conversation state
* Generates appropriate AI responses with tool capabilities
* Maintains state consistency throughout the conversation
* Acts as the primary interface between the user and the ordering system's intelligence

In [17]:
def chatbot_with_tools(state: OrderState) -> OrderState:
    """The chatbot with tools. A simple wrapper around the model's own chat interface."""
    defaults = {"order": [], "finished": False}

    if state["messages"]:
        new_output = llm_with_tools.invoke([BARISTABOT_SYSINT] + state["messages"])
    else:
        new_output = AIMessage(content=WELCOME_MSG)

    # Set up some defaults if not already set, then pass through the provided state,
    # overriding only the "messages" field.
    return defaults | state | {"messages": [new_output]}


# The evaluator_node Function: Quality Control for BaristaBot Responses

This function implements an AI-powered quality assurance system that checks and potentially improves BaristaBot's responses before they reach the customer:

## Function Purpose
The evaluator_node function serves as a quality control checkpoint that intercepts and evaluates every AI-generated response before it reaches the customer. It uses the same LLM technology to evaluate and improve BaristaBot's responses, ensuring they meet quality standards.

## Implementation Details

1. **Selective Evaluation**
   * Only processes states that contain messages
   * Only evaluates messages from the AI (ignores user messages)
   * Returns the state unchanged if there's nothing to evaluate

2. **Evaluation System**
   * Uses the same Google Gemini model for evaluation ("gemini-2.0-flash")
   * Creates a specialized evaluation prompt with specific quality criteria
   * The evaluation system acts as a "second opinion" on the AI's initial response

3. **Quality Criteria**
   The evaluator checks if the response:
   * Correctly follows menu options - Ensures accuracy about available items
   * Uses appropriate tools - Verifies proper use of order management functions
   * Stays on-topic - Maintains focus on cafe-related conversation
   * Is helpful and clear - Ensures customer-friendly communication

4. **Correction Mechanism**
   * If the evaluator returns "GOOD", the original response passes through unchanged
   * If corrections are needed, the evaluator provides a fixed version
   * The function cleans up any meta-text like "Corrected Text:" prefixes
   * Creates a new message with the corrected content
   * Preserves any tool calls from the original message (important for function execution)
   * Updates the state with the corrected message

## System Architecture Impact

This evaluation node adds a critical layer of reliability to the BaristaBot by:
* Reducing errors - Catching and correcting mistakes before they reach customers
* Ensuring consistency - Maintaining accurate information about menu items
* Improving customer experience - Delivering clearer, more helpful responses
* Preserving functionality - Maintaining tool calls even when content is corrected

The node demonstrates a powerful pattern in LLM-based applications where one AI system (the evaluator) reviews and improves the output of another (the primary chatbot), creating a more robust customer-facing experience.

In [18]:
def evaluator_node(state):
    """
    Evaluator that checks BaristaBot responses before they reach the user.
    Simply evaluates and potentially improves responses.
    """
    # Only evaluate if there are messages
    if not state.get("messages", []):
        return state
    
    # Get the last message
    last_msg = state["messages"][-1]
    
    # Only evaluate AI messages (from the bot)
    if not isinstance(last_msg, AIMessage):
        return state
    
    # Create evaluator using the same model
    eval_model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    
    # Create evaluation prompt
    eval_prompt = [
        ("system", """
        You are evaluating a cafe ordering chatbot response. Check if the response:
        1. Correctly follows the menu options
        2. Uses appropriate tools for ordering
        3. Stays on-topic about cafe items only
        4. Is helpful and clear for customers
        
        If the response is good, return "GOOD".
        If the response needs correction, provide ONLY the corrected text without any prefix.
        Do not include phrases like "Corrected Text:" in your response.
        """),
        ("user", f"Evaluate this BaristaBot response: {last_msg.content}")
    ]
    
    # Get evaluation
    evaluation = eval_model.invoke(eval_prompt)
    
    # Apply corrections if needed
    if not evaluation.content.startswith("GOOD"):
        # Process the corrected content to remove any unwanted prefixes
        corrected_content = evaluation.content
        if "Corrected Text:" in corrected_content:
            corrected_content = corrected_content.replace("Corrected Text:", "").strip()
            
        # Create corrected message
        corrected_msg = AIMessage(content=corrected_content)
        
        # Preserve tool calls if present
        if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
            corrected_msg.tool_calls = last_msg.tool_calls
        
        # Replace the message
        return {**state, "messages": state["messages"][:-1] + [corrected_msg]}
    
    # No changes needed
    return state

# Building the BaristaBot Graph: Defining Nodes and Connections

This code creates the conversation flow structure for the BaristaBot application using LangGraph's state graph architecture.

## Tool Organization and Binding

The code organizes the BaristaBot's tools into two distinct categories:

1. **Informational Tools** (`auto_tools`):
   * `get_menu`: Retrieves the full menu
   * `search_menu`: Searches for specific menu information
   * These tools are wrapped in a ToolNode for automatic handling

2. **Order Management Tools** (`order_tools`):
   * Tools that modify the order state (add, confirm, get, clear, place)
   * These require custom implementation in the `order_node`

All tools are bound to the language model with `llm.bind_tools()`, making them available for the LLM to call during conversation.

## Graph Node Definition

The code defines five specialized nodes in the conversation graph:

* **"chatbot"**: Generates AI responses using the tool-enabled LLM
* **"human"**: Captures and processes user input
* **"tools"**: Handles informational tool execution
* **"ordering"**: Implements order state management
* **"evaluator"**: Quality-checks AI responses before they reach the user

## Connection and Flow Definition

The connections between nodes define the possible paths through the conversation:

### Quality Control Flow
* AI responses always flow through the evaluator first (chatbot → evaluator)
* The evaluator then routes to appropriate nodes based on message content

### Dynamic Routing
* `maybe_route_to_tools` determines where to route based on message content
* `maybe_exit_human_node` checks if the user wants to exit

### Tool Processing Loops
* Both tool types route back to the chatbot after execution
* This creates conversation loops for order building

### Conversation Entry Point
* The graph begins at the chatbot node (START → chatbot)

## Compilation
The `compile()` method creates the executable workflow.

This architecture creates a flexible conversation flow that dynamically adapts based on user input and AI responses, while maintaining proper state management throughout the ordering process.

In [19]:
# Define the tools and create a "tools" node.
auto_tools = [get_menu, search_menu]
tool_node = ToolNode(auto_tools)
order_tools = [add_to_order, confirm_order, get_order, clear_order, place_order]
llm_with_tools = llm.bind_tools(auto_tools + order_tools)

graph_builder = StateGraph(OrderState)

# Nodes
graph_builder.add_node("chatbot", chatbot_with_tools)
graph_builder.add_node("human", human_node)
graph_builder.add_node("tools", tool_node)
graph_builder.add_node("ordering", order_node)
graph_builder.add_node("evaluator", evaluator_node)

# Chatbot -> {ordering, tools, human, END}
# 3. Add a simple edge from chatbot to evaluator
graph_builder.add_edge("chatbot", "evaluator")
# 3. Connect evaluator to same destinations as chatbot originally went to
graph_builder.add_conditional_edges("evaluator", maybe_route_to_tools)
# Human -> {chatbot, END}
graph_builder.add_conditional_edges("human", maybe_exit_human_node)

# Tools (both kinds) always route back to chat afterwards.
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("ordering", "chatbot")

graph_builder.add_edge(START, "chatbot")
graph_with_order_tools = graph_builder.compile()


# Application Execution: Starting the BaristaBot

This single line of code represents the execution entry point for the entire BaristaBot application:

```python
state = graph_with_order_tools.invoke({"messages": []}, config)
```

## What This Line Does

This compact but powerful line of code:

### Initializes the Conversation:
- Starts with an empty message list `{"messages": []}`
- The empty initial state serves as a blank slate for the conversation

### Triggers the Conversation Flow:
- Invokes the compiled state graph (`graph_with_order_tools`)
- Starts at the designated START node (which routes to "chatbot")
- Executes the welcome message generation

### Configures Runtime Behavior:
- Applies the config settings (defined earlier as `{"recursion_limit": 100}`)
- The recursion limit of 100 allows for complex, multi-turn conversations
- This prevents infinite loops while supporting lengthy ordering processes

### Returns the Final State:
- After the conversation completes, the final state is stored in the `state` variable
- This contains the complete order and conversation history

## Execution Flow

When this line runs:
1. The chatbot node generates the welcome message
2. The evaluator checks the message quality
3. Control passes to the human node for user input
4. Based on user responses, the flow navigates through various nodes
5. The conversation continues until either:
   - The user enters an exit command
   - An order is successfully placed

This single line effectively sets in motion the entire conversation loop, with the state being continuously updated and passed between nodes as the interaction progresses.

In [20]:
state = graph_with_order_tools.invoke({"messages": []}, config)

Model: Welcome to the BaristaBot cafe. Type `q` to quit. How may I serve you today?


User:  q


In [21]:
pprint(state["order"])

[]
